In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

In [18]:
df = pd.read_csv('data/diabetes.csv', dtype=float)

In [23]:
X = df.drop('Outcome', axis=1).values
y = df['Outcome']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
class DiabetesDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.float32)

  def __len__(self):
    return len(self.features)
  
  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

In [46]:
train_dataset = DiabetesDataset(X_train, y_train)
test_dataset = DiabetesDataset(X_test, y_test)

In [47]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [36]:
class DiabetesClassifier(nn.Module):
  def __init__(self):
    super(DiabetesClassifier, self).__init__()
    self.fc1 = nn.Linear(8, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))
    return x

In [43]:
model = DiabetesClassifier()

In [44]:
# loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [55]:
epochs = 10
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  
  for inputs, labels in train_loader:
    optimizer.zero_grad() # zero the parameter gradients
    outputs = model(inputs) # forward pass
    loss = criterion(outputs, labels.unsqueeze(1))
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  
  print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss/len(train_loader)}')

Epoch 1/10, Loss: 0.5261257857084274
Epoch 2/10, Loss: 0.5249041602015495
Epoch 3/10, Loss: 0.5387098073959351
Epoch 4/10, Loss: 0.5764228746294975
Epoch 5/10, Loss: 0.5739735543727875
Epoch 6/10, Loss: 0.5387895837426185
Epoch 7/10, Loss: 0.5389506310224533
Epoch 8/10, Loss: 0.504512856900692
Epoch 9/10, Loss: 0.5618639215826988
Epoch 10/10, Loss: 0.5514127910137177


In [56]:
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    predicted_labels = (outputs >= 0.5).float()
    total_correct += (predicted_labels == labels.unsqueeze(1)).sum()
    total_samples += labels.size(0)
  
accuracy = total_correct / total_samples
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.73
